In [ ]:
# ================================
# Perceptron animation showing misclassified points
# ================================
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display
import pandas as pd

# ------------------------------
# 1. Generate 2D linearly separable data
# ------------------------------
rng = np.random.RandomState(0)
n_per_class = 20
X_pos = rng.randn(n_per_class, 2) + np.array([1.0, 1.0])
X_neg = rng.randn(n_per_class, 2) + np.array([-1.0, -1.0])

In [7]:
X = np.vstack([X_pos, X_neg])
y = np.hstack([np.ones(n_per_class), -np.ones(n_per_class)])
perm = rng.permutation(len(y))
X = X[perm]
y = y[perm]
X_aug = np.hstack([X, np.ones((X.shape[0], 1))])  # add bias

# ------------------------------
# 2. Train perceptron and record snapshots
# ------------------------------
def sign(v):
    return 1 if v >= 0 else -1

w = np.zeros(3)
eta = 1.0
max_epochs = 20
snapshots = []
step = 0

for epoch in range(1, max_epochs+1):
    errors = 0
    for i in range(len(y)):
        xi, yi = X_aug[i], y[i]
        y_pred = sign(np.dot(w, xi))
        # Identify misclassified points
        misclassified = (y_pred != yi)
        if misclassified:
            w = w + eta * yi * xi
            step += 1
            snapshots.append({
                "step": step,
                "epoch": epoch,
                "index": i,
                "x0": X[i,0],
                "x1": X[i,1],
                "y": yi,
                "w0": w[0],
                "w1": w[1],
                "b": w[2],
                "misclassified_indices": np.where([sign(np.dot(w, X_aug[j])) != y[j] for j in range(len(y))])[0]
            })
            errors += 1
    if errors == 0:
        break

# ------------------------------
# 3. Animation setup
# ------------------------------
x_min, x_max = X[:,0].min()-1, X[:,0].max()+1
y_min, y_max = X[:,1].min()-1, X[:,1].max()+1
xs_line = np.linspace(x_min, x_max, 200)

def decision_boundary_points(wvec):
    w0, w1, b = wvec
    if abs(w1) > 1e-8:
        ys = -(w0 * xs_line + b) / w1
        return xs_line, ys
    elif abs(w0) > 1e-8:
        x_vert = -b / w0
        return np.array([x_vert, x_vert]), np.array([y_min, y_max])
    else:
        return np.array([]), np.array([])

fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_xlabel("x0")
ax.set_ylabel("x1")
ax.set_title("Perceptron training with misclassified points")

# Scatter for original points
scatter_pos = ax.scatter(X[y==1,0], X[y==1,1], marker='o', color='blue', label='Positive')
scatter_neg = ax.scatter(X[y==-1,0], X[y==-1,1], marker='s', color='red', label='Negative')
boundary_line, = ax.plot([], [], 'k-', lw=2)
updated_marker = ax.scatter([], [], marker='x', s=150, color='green', label='Updated point')
mis_marker = ax.scatter([], [], marker='o', s=120, facecolors='none', edgecolors='orange', label='Misclassified')

ax.legend(loc='upper left')

def init():
    boundary_line.set_data([], [])
    updated_marker.set_offsets(np.empty((0,2)))
    mis_marker.set_offsets(np.empty((0,2)))
    return boundary_line, updated_marker, mis_marker

def update(frame_idx):
    snap = snapshots[frame_idx]
    wvec = np.array([snap["w0"], snap["w1"], snap["b"]])
    xs, ys = decision_boundary_points(wvec)
    boundary_line.set_data(xs, ys)
    
    # Highlight the point being updated
    updated_marker.set_offsets([[snap["x0"], snap["x1"]]])
    
    # Highlight all currently misclassified points
    mis_idx = snap["misclassified_indices"]
    mis_marker.set_offsets(X[mis_idx])
    
    ax.set_title(f"Step {snap['step']} Epoch {snap['epoch']} Updated idx {snap['index']}")
    return boundary_line, updated_marker, mis_marker

anim = animation.FuncAnimation(fig, update, init_func=init,
                               frames=len(snapshots), interval=500, blit=False)
plt.close(fig)
display(HTML(anim.to_jshtml()))

# ------------------------------
# 4. Snapshot table
# ------------------------------
snap_df = pd.DataFrame(snapshots)
display(snap_df.head(10))

,step,epoch,index,x0,x1,y,w0,w1,b,misclassified_indices
0,1,1,0,-2.706270,0.950775,-1.0,2.706270,-0.950775,-1.0,"[3, 11, 16, 28, 31]"
1,2,1,3,-1.552990,1.653619,1.0,1.153280,0.702843,0.0,"[3, 9]"
2,3,1,9,0.112214,-0.980796,1.0,1.265495,-0.277953,1.0,"[1, 3, 25, 27, 28, 30]"
3,4,1,25,-0.270909,-0.871017,-1.0,1.536404,0.593064,0.0,"[3, 9]"
4,5,2,3,-1.552990,1.653619,1.0,-0.016586,2.246682,1.0,"[0, 9, 12, 24]"
5,6,2,9,0.112214,-0.980796,1.0,0.095629,1.265886,2.0,"[0, 1, 2, 5, 12, 13, 14, 19, 20, 25, 30, 33, 3..."
6,7,2,12,-2.252795,-0.222510,-1.0,2.348424,1.488396,1.0,"[3, 9]"
7,8,3,3,-1.552990,1.653619,1.0,0.795434,3.142014,2.0,"[0, 9]"
8,9,3,9,0.112214,-0.980796,1.0,0.907648,2.161218,3.0,"[0, 1, 2, 12, 25]"
9,10,3,12,-2.252795,-0.222510,-1.0,3.160444,2.383727,2.0,[]
